In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.07718825340270996
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 17.887776340214057
200000 30.685327808465335
300000 31.96300038274984
400000 35.921436350511165
500000 37.71434803104374
600000 39.0132872520093
700000 32.744628062084665
800000 34.38315144542056
900000 37.14177179802134
1000000 36.70022289002868
1100000 13.035811238903783
1200000 30.08745778570959
1300000 36.458744311325454
1400000 36.9992709828153
1500000 33.72884467314429
1600000 34.20019443403254
1700000 39.32274234423241
1800000 36.39731484642856
1900000 34.824847107692484
2000000 32.245970387573585
2100000 29.84661306002208
2200000 8.141159998230371
2300000 36.604292415531695
2400000 37.572807293025626
2500000 38.95850783385996
2600000 36.775252330348174
2700000 39.553344020369046
2800000 34.896246748154304
2900000 37.741795690651585
3000000 35.45326

29700000 34.32036792711709
29800000 29.362063861502087
29900000 17.493116305660596
30000000 33.61008539252314
30100000 31.1982431310227
30200000 32.896999634124285
30300000 20.74728460515852
30400000 24.50531203402685
30500000 33.105524920924516
30600000 31.821065809617835
30700000 32.163342932707245
30800000 32.87038971181425
30900000 28.023548877426645
31000000 31.41118559598074
31100000 24.97858722636493
31200000 12.735736630825402
31300000 30.59285976383326
31400000 27.36226452057919
31500000 31.115313571549052
31600000 29.77721399391868
31700000 30.82715233713844
31800000 34.27812058975936
31900000 29.591114064543103
32000000 15.30335613763168
32100000 31.21374386819171
32200000 32.797650546321094
32300000 31.88514895046863
32400000 32.1205958730048
32500000 11.653834012927762
32600000 31.865888298611605
32700000 33.660856715730915
32800000 32.1037629839331
32900000 30.15502365941368
33000000 29.700999373837163
33100000 30.50345045071808
33200000 27.718918238545474
33300000 12.092

59500000 26.293414011703298
59600000 27.161768758503957
59700000 13.797449088369152
59800000 29.368577873075115
59900000 30.23933991187213
60000000 28.3037909525589
60100000 30.679108861518287
60200000 28.215445177473743
60300000 30.186239456594862
60400000 28.19257871906691
60500000 28.367571236570114
60600000 16.911399930585233
60700000 21.757745793418287
60800000 27.797548337265017
60900000 24.952517436412965
61000000 24.806940217530556
61100000 25.88780543274411
61200000 12.738239881843903
61300000 21.101535038342714
61400000 26.27579604356951
61500000 24.44656453496261
61600000 25.719949821782237
61700000 24.52694650254561
61800000 15.034823822859138
61900000 25.584801710810094
62000000 27.124516050233506
62100000 28.9312816548972
62200000 27.50031273988999
62300000 29.386263584520155
62400000 29.342922622554912
62500000 29.35936583460622
62600000 28.030924156053963
62700000 29.819625969982333
62800000 10.684380511976267
62900000 25.848231338164094
63000000 26.55029084017401
63100

89200000 18.93730740289342
89300000 24.27231261723062
89400000 23.324825762614854
89500000 11.472031565654662
89600000 24.168654516712515
89700000 22.636980381444655
89800000 25.12741379953232
89900000 24.051226504596038
90000000 26.282753575470867
90100000 25.285987746064315
90200000 25.68341568154526
90300000 24.925877877160087
90400000 25.885738370514474
90500000 25.10632083811913
90600000 23.696954736192577
90700000 24.257613442192607
90800000 21.425606105148894
90900000 10.788374646168464
91000000 22.417437944316607
91100000 22.802835448867054
91200000 21.696767591686907
91300000 10.107808417350665
91400000 22.366589932327372
91500000 22.003202411588113
91600000 18.271816894962463
91700000 17.420038331869033
91800000 22.342607402317476
91900000 23.588038745554556
92000000 22.074538468984994
92100000 26.482552117150288
92200000 24.682527413261084
92300000 25.402597730347342
92400000 26.285824559919586
92500000 26.089948197635994
92600000 25.454942934845835
92700000 26.8572413133147

118200000 24.8902078257433
118300000 23.834383689277615
118400000 21.35863461292858
118500000 23.554759489980437
118600000 20.5352604612959
118700000 23.444384862447162
118800000 20.803622173457192
118900000 22.040445401390222
119000000 13.909048395295173
119100000 16.622177113369954
119200000 7.024254761179303
119300000 14.991080220814226
119400000 16.873376265619314
119500000 22.005400464279504
119600000 21.649628860589317
119700000 21.75403147126907
119800000 23.239935016933835
119900000 21.23015414757484
120000000 22.42195164978703
120100000 22.739882986309038
120200000 24.899844790632027
120300000 24.57497768438823
120400000 24.216523558788747
120500000 24.33501956597915
120600000 21.081192116656105
120700000 23.952827462224676
120800000 21.527368287739243
120900000 22.844334922284112
121000000 22.215467570225815
121100000 19.846546794847452
121200000 9.947752864634662
121300000 12.93457264503753
121400000 14.041389045414412
121500000 14.343045412955927
121600000 20.63876340995687

...

290300000 5.527686779898709
290400000 5.241690861628161
290500000 5.142128989777839
290600000 5.003240990166181
290700000 4.696978053510969
290800000 4.2204024780316916
290900000 1.0749285351817848
291000000 1.1037659408621503
291100000 5.802895272023284
291200000 4.644870553730712
291300000 5.13940219483822
291400000 5.760476927933477
291500000 4.835855983941621
291600000 4.7327563445274485
291700000 4.6550735217152175
291800000 5.88006516576249
291900000 4.9108032909471895
292000000 5.983652663763615
292100000 5.244104436262481
292200000 5.3711793319576975
292300000 5.2247278047764825
292400000 4.6703909150631135
292500000 5.187635252714757
292600000 5.158079399055971
292700000 5.229675644810308
292800000 4.759210379117489
292900000 5.208665915008676
293000000 1.0860212752785823
293100000 1.0873651434350624
293200000 4.132298985313463
293300000 4.687237017831999
293400000 4.971513752777299
293500000 5.054442010793064
293600000 4.874654947082464
293700000 4.7365542590795515
293800000 

319300000 3.2213236712464983
319400000 3.3298582656967106
319500000 2.701250447554268
319600000 2.541404535065911
319700000 2.0357429620454863
319800000 2.0881383166348964
319900000 2.56119495795514
320000000 3.1901146359217942
320100000 2.850400481634053
320200000 2.228356997880217
320300000 2.6631055454694805
320400000 2.490805856675608
320500000 2.2640777009261392
320600000 2.3861658424056134
320700000 3.5048399370796535
320800000 3.975037433313565
320900000 0.65994949265035
321000000 0.6014305208270417
321100000 2.119107654310713
321200000 3.1945649955312785
321300000 2.3336634688538487
321400000 3.923517363466523
321500000 2.35703127211082
321600000 3.2466109551487032
321700000 3.9345816306876205
321800000 2.6072048469327287
321900000 2.4336706227500726
322000000 2.619372426171994
322100000 3.0267391675185795
322200000 2.6038326889951264
322300000 2.499814421403959
322400000 2.3246968751356065
322500000 3.276660887530346
322600000 2.42186771607991
322700000 2.460124427560204
32280

347900000 1.4128528088440757
348000000 1.329208235356009
348100000 1.3693357865743618
348200000 1.251597004394877
348300000 1.207049120268611
348400000 1.2303841925422834
348500000 1.2966904467939298
348600000 1.7824397629348296
348700000 1.3407470269530741
348800000 0.4997234555185895
348900000 0.3248877609943966
349000000 0.7763615909963639
349100000 2.5867075355157616
349200000 1.4196659387857946
349300000 1.276369900015608
349400000 1.3233846852436641
349500000 1.1554845121805362
349600000 1.9475895033615336
349700000 1.3770691191135902
349800000 1.3486295266978152
349900000 1.413147092008967
350000000 1.4754683318678679
350100000 1.3200221009773307
350200000 1.2612274468379432
350300000 1.2333377800260266
350400000 1.2017886730080591
350500000 2.2412240019890555
350600000 1.277932708162425
350700000 1.279775239749905
350800000 1.2279884358203483
350900000 0.9884880542042315
351000000 0.401548046687995
351100000 0.5444854749811676
351200000 2.622070601371435
351300000 1.60964468377

375900000 0.30178284477752715
376000000 0.29950933815705316
376100000 0.30270022977868316
376200000 0.30215957847046127
376300000 0.31626767670039074
376400000 0.30308825283589763
376500000 0.30117404714370366
376600000 0.299916243083281
376700000 0.2992172439568401
376800000 0.2983459550788323
376900000 0.29851970963065755
377000000 0.29775394693105817
377100000 0.296380944443663
377200000 0.2952061900229308
377300000 0.2978631817421297
377400000 0.2978968762329585
377500000 0.29366511749006646
377600000 0.2932774243832509
377700000 0.2922339988379432
377800000 0.2959380779100603
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 1.9802439605649829
378000000 2.0091841738791505
378100000 1.9803350099163968
378200000 2.2413368172251364
378300000 1.999181603544359
378400000 1.8858814365162717
378500000 1.9485673690509222
378600000 2.38358628147182
378700000 1.9102275837688276
378800000 1.823843558447558
378900000 2.1423967284388676
379000000 2.08966511577822

404000000 0.7751928578416586
404100000 0.6264513273594572
404200000 0.7341075443752276
404300000 0.7209777079423944
404400000 0.6615953614938723
404500000 0.6697423378479912
404600000 0.5993041346669833
404700000 0.7456657718277097
404800000 0.6447063079262124
404900000 0.6828648711395741
405000000 0.7329752106393158
405100000 0.7398868037232221
405200000 0.6816278101705154
405300000 0.6123152169058065
405400000 0.6823599510205779
405500000 0.5593488494292557
405600000 0.6620203188955273
405700000 0.7168136895513447
405800000 0.6057904595555902
405900000 0.6426403504789749
406000000 0.5468137630048043
406100000 0.6788127938371977
406200000 0.6560290648611651
406300000 0.5888896066775726
406400000 0.6629693852689842
406500000 0.6559909365464324
406600000 0.5907039345589625
406700000 0.5014933111254574
406800000 0.48423865861942833
406900000 0.6313302355216714
407000000 0.5929751315436125
407100000 0.5495985136612342
407200000 0.7235346736896898
407300000 0.5276075716481685
407400000 0.5